In [ ]:
__author__ = "Tim Cuddeback"

import yfpy, json
from yfpy.query import YahooFantasySportsQuery
import pandas as pd
pd.set_option('display.max_colwidth', None)

# Function that pulls data from Yahoo's Fantasy API and returns a sorted list of tuples with (team_id,current points)
league_id = "12514" # put your real league id here
game_id = "399" # put the game id here (game id's reflect the type of sport and the year)
game_code = "nfl" # put the game code here
season = "2020" # put the year of the current fantasy season
auth_dir = '' # put the location where you are storing the client_id/secret
working_dir = '' # put the location where you want excel file to be saved

# Query yfpy to get data
# yahoo_query = YahooFantasySportsQuery(auth_dir, league_id, game_id=game_id, game_code=game_code, offline=False)

In [ ]:
# Get standings data to get the points for the day
# data = []
# for i in range (0,17):
#     d = yahoo_query.get_league_matchups_by_week(i)
#     data.append(d)
# d = yahoo_query.get_league_draft_results()
# t = yahoo_query.get_league_teams()

In [ ]:
# teams = []
# for i in range(0,len(t)):
#     teams.append(t[i]['team'])
# df = pd.DataFrame.from_dict(teams)
# df = df[[0,2]]
# df['Year'] = season
# df.to_csv('lead-teams.csv',mode='a',header=False,index=False)
# d2 = []
# for i in range(0,len(d)):
#     d2.append(d[i]['draft_result'])
# df = pd.DataFrame.from_dict(d2)
# df['Year'] = season
# df.to_csv('league-draft.csv',mode='a',header=False,index=False)
# data2 = []
# for j in range(0,len(data)):
#     for i in range(0,len(data[j])):
#         data2.append(json.loads(str(data[j][i]['matchup'])))
# for x in range(0,len(data2)):
#     try:
#         row = {"week" : data2[x]['week'],
#            "week_start" : data2[x]['week_start'],
#            "team_id1" : data2[x]['teams'][0]['team']['managers']['manager']['manager_id'],
#            "owner1" : data2[x]['teams'][0]['team']['managers']['manager']['nickname'],
#            "team1" : data2[x]['teams'][0]['team']['name'],
#            "points1" : data2[x]['teams'][0]['team']['team_points']['total'],
#            "team_id2" : data2[x]['teams'][1]['team']['managers']['manager']['manager_id'],
#            "owner2" : data2[x]['teams'][1]['team']['managers']['manager']['nickname'],
#            "team2" : data2[x]['teams'][1]['team']['name'],
#            "points2" : data2[x]['teams'][1]['team']['team_points']['total']}
#     except:
#         row = {"week" : data2[x]['week'],
#            "week_start" : data2[x]['week_start'],
#            "team_id1" : data2[x]['teams'][0]['team']['managers']['manager']['manager_id'],
#            "owner1" : data2[x]['teams'][0]['team']['managers']['manager']['nickname'],
#            "team1" : data2[x]['teams'][0]['team']['name'],
#            "points1" : data2[x]['teams'][0]['team']['team_points']['total'],
#            "team_id2" : data2[x]['teams'][1]['team']['managers'][0]['manager']['manager_id'],
#            "owner2" : data2[x]['teams'][1]['team']['managers'][0]['manager']['nickname'],
#            "team2" : data2[x]['teams'][1]['team']['name'],
#            "points2" : data2[x]['teams'][1]['team']['team_points']['total']}
#     try:
#         row1 = {"week" : data2[x]['week'],
#            "week_start" : data2[x]['week_start'],
#            "team_id1" : data2[x]['teams'][1]['team']['managers']['manager']['manager_id'],
#            "owner1" : data2[x]['teams'][1]['team']['managers']['manager']['nickname'],
#            "team1" : data2[x]['teams'][1]['team']['name'],
#            "points1" : data2[x]['teams'][1]['team']['team_points']['total'],
#            "team_id2" : data2[x]['teams'][0]['team']['managers']['manager']['manager_id'],
#            "owner2" : data2[x]['teams'][0]['team']['managers']['manager']['nickname'],
#            "team2" : data2[x]['teams'][0]['team']['name'],
#            "points2" : data2[x]['teams'][0]['team']['team_points']['total']}
#     except:
#         row1 = {"week" : data2[x]['week'],
#            "week_start" : data2[x]['week_start'],
#            "team_id1" : data2[x]['teams'][1]['team']['managers'][0]['manager']['manager_id'],
#            "owner1" : data2[x]['teams'][1]['team']['managers'][0]['manager']['nickname'],
#            "team1" : data2[x]['teams'][1]['team']['name'],
#            "points1" : data2[x]['teams'][1]['team']['team_points']['total'],
#            "team_id2" : data2[x]['teams'][0]['team']['managers']['manager']['manager_id'],
#            "owner2" : data2[x]['teams'][0]['team']['managers']['manager']['nickname'],
#            "team2" : data2[x]['teams'][0]['team']['name'],
#            "points2" : data2[x]['teams'][0]['team']['team_points']['total']}
#     df = pd.DataFrame.from_dict([row,row1])
#     df.to_csv('league-data2.csv',mode='a',index=False,header=False)